In [ ]:
import csv 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

with open('Data/2015.tsv','r') as tsv:
    data = [line.strip().split('\t') for line in tsv]
    
col_names = data[0]
df = pd.DataFrame(data, columns=col_names)
df = df[1:]

#open code_sex.csv and convert it to a DataFrame
with open('Data/code_sex.csv','r') as csv:
    sex_descrip = [line.strip().split(',') for line in csv]

sex_header = sex_descrip[0]
df_sex = pd.DataFrame(sex_descrip, columns=sex_header)
df_sex = df_sex[1:]
indexed_df_sex = df_sex.set_index('Code')

#open code_body_part.csv and convert it to a DataFrame
with open('Data/code_body_part.csv','r') as csv:
    body_part_descrip = [line.strip().split(',') for line in csv]

body_part_header = body_part_descrip[0]
df_body_part = pd.DataFrame(body_part_descrip, columns=body_part_header)
df_body_part = df_body_part[1:]
indexed_df_body_part = df_body_part.set_index('Code')

#open code_diagnosis.csv and convert it to a DataFrame
with open('Data/code_diagnosis.csv','r') as csv:
    diagnosis_descrip = [line.strip().split(',') for line in csv]

diagnosis_header = diagnosis_descrip[0]
df_diagnosis = pd.DataFrame(diagnosis_descrip, columns=diagnosis_header)
df_diagnosis = df_diagnosis[1:]
indexed_df_diagnosis = df_diagnosis.set_index('Code')

#open code_disposition.csv and convert it to a DataFrame
with open('Data/code_disposition.csv','r') as csv:
    disposition_descrip = [line.strip().split(',') for line in csv]

disposition_header = disposition_descrip[0]
df_disposition = pd.DataFrame(disposition_descrip, columns=disposition_header)
df_disposition = df_disposition[1:]
indexed_df_disposition = df_disposition.set_index('Code')

#open code_fire.csv and convert it to a DataFrame
with open('Data/code_fire.csv','r') as csv:
    fire_descrip = [line.strip().split(',') for line in csv]

fire_header = fire_descrip[0]
df_fire = pd.DataFrame(fire_descrip, columns=fire_header)
df_fire = df_fire[1:]
indexed_df_fire = df_fire.set_index('Code')

#open code_locale.csv and convert it to a DataFrame
with open('Data/code_locale.csv','r') as csv:
    locale_descrip = [line.strip().split(',') for line in csv]

locale_header = locale_descrip[0]
df_locale = pd.DataFrame(locale_descrip, columns=locale_header)
df_locale = df_locale[1:]
indexed_df_locale = df_locale.set_index('Code')

#open code_product.csv and convert it to a DataFrame
with open('Data/code_product.csv','r') as csv:
    product_descrip = [line.strip().split(',',1) for line in csv]

product_header = product_descrip[0]
df_product = pd.DataFrame(product_descrip, columns=product_header)
df_product = df_product[1:]
indexed_df_product = df_product.set_index('Code')

#open code_race.csv and convert it to a DataFrame
with open('Data/code_race.csv','r') as csv:
    race_descrip = [line.strip().split(',') for line in csv]

race_header = race_descrip[0]
df_race = pd.DataFrame(race_descrip, columns=race_header)
df_race = df_race[1:]
indexed_df_race = df_race.set_index('Code')

#merge all the codes
df['sex_descrip']=df.sex.map(indexed_df_sex.Description)
df['body_part_descrip']=df.body_part.map(indexed_df_body_part.Description)
df['diag_descrip']=df.diag.map(indexed_df_diagnosis.Description)
df['disposition_descrip']=df.disposition.map(indexed_df_disposition.Description)
df['fire_descrip']=df.fmv.map(indexed_df_fire.Description)
df['locale_descrip']=df.location.map(indexed_df_locale.Description)
df['product1_descrip']=df.prod1.map(indexed_df_product.Description)
df['product2_descrip']=df.prod2.map(indexed_df_product.Description)
df['race_descrip']=df.race.map(indexed_df_race.Description)

#combine the 2 narr fields into 1
df['Notes'] = df['narr1'].map(str) + df['narr2'].map(str)

df1 = df[['CPSC Case #', 
         'trmt_date', 
         'psu', 
         'weight', 
         'age', 
         'sex_descrip', 
         'race_descrip',
         'body_part_descrip', 
         'diag_descrip', 
         'disposition_descrip', 
         'fire_descrip', 
         'locale_descrip', 
         'product1_descrip', 
         'product2_descrip', 
         'Notes']]

#convert numerical columns to numeric data type
df1 = df1.apply(lambda x: pd.to_numeric(x,errors="ignore"))

#convert 'trmt_date' column to datetime type
df1['trmt_date'] = df1['trmt_date'].apply(lambda x: pd.to_datetime(x,format='%m/%d/%Y'))

#adjust ages >120yo
def age_filter(age):
    if age > 120:
        return 1
    else:
        return age

df1['age'] = df1['age'].astype(int).map(age_filter)

#extra functions
def textfind(dataframe, field, string):
    """ Takes dataframe, field (column in the dataframe), and a string to search
    Allow for use of regular expressions.
    Returns a dataframe of the records where the string is found"""
    import re
    a = [df1.loc[i] for i in range(1,len(dataframe)) \
         if re.search(string, dataframe[field][i])]
    return pd.DataFrame(a)
    
def notereader(dataframe):
    """ Takes a dataframe that contains the 'Notes' field and prints it out in a
    readable fashion.
    Returns a dictionary of Index (integer): Note (string) pairs"""
    output_dict ={}
    for i in dataframe.index:           
        output_dict[i] = dataframe.Notes[i]
        print ("INDEX", i,"\n", dataframe.Notes[i],"\n")
    return output_dict

# DRUGS V ALCOHOL V POT

In [ ]:
#drugs

#value counts by age containing cocaine/heroin
doped_af = df1[df1['Notes'].str.contains('COCAINE|HEROIN')]
age_doped = doped_af[doped_af['age'].value_counts(sort = False)]

#bin ages into groups of 10 years and sums value counts per bin 
bins = np.arange(age_doped.index.min(), age_doped.index.max(), 10)
doped_age_groups = age_doped.groupby(pd.cut(age_doped.index, bins))
drug_graph = doped_age_groups.sum()
drug_graph.plot(kind = 'bar')


# Using Omar's pretty histographs
# M_doped = doped_af[doped_af.sex_descrip == 'MALE'].age
# F_doped = doped_af[doped_af.sex_descrip == 'FEMALE'].age
# Mdoped = plt.hist(doped_M_bar,alpha=0.5, bins = 14, color = 'blue',label='MALE')
# Fdoped = plt.hist(doped_F_bar,alpha=0.5, bins = 10,color = 'red',label='FEMALE')


In [ ]:
#value counts by age containing alcohol-related terms
drunk_af = df1[df1['Notes'].str.contains('DRUNK|DRINKING|ALCOHOL|INEBRIATED|INTOXICATED')]
drunk_af = drunk_af['age'].value_counts(sort = False)

#bin ages
drunk_age_groups = drunk_af.groupby(pd.cut(drunk_af.index, bins))
drunk_graph = drunk_age_groups.sum()
drunk_graph.plot(kind = 'bar')

#for labeling bar graphs
# drunk_af_graph.set_xlabel("Ages")
# drunk_af_graph.set_ylabel("Number of people")


In [ ]:
#value counts by age containing marijuana-related terms

high_filter = df1[df1['Notes'].str.contains('HIGH|MARIJUANA')]

high_filter = high_filter[high_filter['Notes'].str.contains('HEELED|CARBON|MILAGE|HIGHWAY|TOO|FX') == False]
high_filter = high_filter[high_filter['Notes'].str.contains('CHAIR|BP|FIVE|ANKLE|SHOT|SWING') == False]
high_filter = high_filter[high_filter['Notes'].str.contains('HEELS|FLOWER') == False]
high_filter = high_filter[high_filter['Notes'].str.contains('THIGH|POTATOES|POTHOLE|HOLE|SPILL|STOVE') == False]
high_filter = high_filter[high_filter['Notes'].str.contains('BED|HIGHEST|HIGHER') == False]
high_filter = high_filter[high_filter['Notes'].str.contains('BAR|HOT|FALL') == False]
high_filter = high_filter[high_filter['Notes'].str.contains('SHELF') == False]
high_filter = high_filter[high_filter['Notes'].str.contains('HEEL') == False]
high_filter = high_filter[high_filter['Notes'].str.contains('STEPS') == False]
high_filter = high_filter[high_filter['Notes'].str.contains('POWERED') == False]
high_filter = high_filter[high_filter['Notes'].str.contains('STOOL') == False]
high_filter = high_filter[high_filter['Notes'].str.contains('FT') == False]
high_filter = high_filter[high_filter['Notes'].str.contains('FEET') == False]
high_filter = high_filter[high_filter['Notes'].str.contains('UP') == False]
high_filter = high_filter[high_filter['Notes'].str.contains('INTENSITY') == False]
high_filter = high_filter[high_filter['Notes'].str.contains('HEEL') == False]
high_filter = high_filter[high_filter['Notes'].str.contains('SCHOOL') == False]
high_filter = high_filter[high_filter['Notes'].str.contains('SPEED') == False]
high_filter = high_filter[high_filter['Notes'].str.contains('PRESSURE') == False]
high_filter = high_filter[high_filter['Notes'].str.contains('CABINET') == False]
high_filter = high_filter[high_filter['Notes'].str.contains('JUMP') == False]
high_af = high_filter[high_filter['Notes'].str.contains('TOP') == False]

#checking filtered out notes
# pd.options.display.max_colwidth = 1000
# high_af['Notes']

#value counts by age containing high-related terms
high_af = high_af['age'].value_counts(sort = False)
high_age_groups = high_af.groupby(pd.cut(high_af.index, bins))
high_graph = high_age_groups.sum()
high_graph.plot(kind = 'bar')

#bar graph separating by ages of injuries related to being high
# high_af_graph = high_af['age'].value_counts().head(10).plot(kind = 'bar')
# high_af_graph.set_xlabel("Ages")
# high_af_graph.set_ylabel("Number of people")

In [ ]:
#combine drunk, marijuana, drugs

#df_influenced= high_graph + drug_graph + drunk_graph
df_influenced = high_graph.join(drug_graph)

In [ ]:
# Time associated 

#sorts injuries by day in chronological order




In [ ]:
# Looking for weight associated injuries

weight = df1['weight'].value_counts().sort_index(ascending = False).head(10)

# From 6 most common weights for injuries, ranking heaviest to lightest weights and showing product description
weight_1 = df1[df1['weight'].str.contains('97.9239')]
dfweight1 = weight_1['product1_descrip'].value_counts().head()

weight_2 = df1[df1['weight'].str.contains('85.7374')]
dfweight2 = weight_2['product1_descrip'].value_counts().head()

weight_3 = df1[df1['weight'].str.contains('83.2157')]
dfweight3 = weight_3['product1_descrip'].value_counts().head()

weight_4 = df1[df1['weight'].str.contains('80.8381')]
dfweight4 = weight_4['product1_descrip'].value_counts().head()

weight_5 = df1[df1['weight'].str.contains('78.5926')]
dfweight5 = weight_5['product1_descrip'].value_counts().head()

weight_6 = df1[df1['weight'].str.contains('74.8813')]
dfweight6 = weight_6['product1_descrip'].value_counts().head()

weight_7 = df1[df1['weight'].str.contains('49.2646')]
dfweight7 = weight_7['product1_descrip'].value_counts().head()

weight_7 = df1[df1['weight'].str.contains('37.6645 ')]
dfweight8 = weight_7['product1_descrip'].value_counts().head()

# stacked bar graph
weight_graph = pd.DataFrame({'97lbs':dfweight1, '85lbs':dfweight2, '83lbs':dfweight3, '80lbs':dfweight4, '78lbs':dfweight5, '75lbs':dfweight6, '49lbs':dfweight7, '37lbs':dfweight8}).plot(kind = 'bar', stacked = True)
weight_graph.set_ylabel('Number of injuries')
weight_graph.set_xlabel('Products causing injury')


In [ ]:
# Girls v Boys
# General injuries 

gvb_general = df1['sex_descrip'].value_counts().plot(kind = "bar")
gvb_general = gvb_general.set_ylabel("Number of injuries")
gvb_general

In [ ]:
# Girls v Boys
# Most common products
    
gvb_products = df1['product1_descrip'].value_counts().head(10).plot(kind = 'bar', legend=None, title = "The most dangerous products for males vs females")
gvb_products.set_xlabel("Product types")
gvb_products.set_ylabel("Frequency of occurences")
gvb_products

# Trying to groupby sex, then 10 most common things for each sex
y = df1.groupby(['sex_descrip', 'product1_descrip'], sort = True).size()

In [ ]:
# Girls v Boys
# Most common products
# 10 most common items, need to break by gender
ten_products = df1['product1_descrip'].value_counts(ascending = False)
ten_products
#ten_products = ten_products.head(10).plot.bar(by = 'product1_descrip')


In [ ]:
# Girls v Boys
# Most commonly injured body part
# Need to change to gvb

body_parts = df1['body_part_descrip'].value_counts().plot(kind = 'bar')
body_parts